In [1]:
library(RPostgreSQL)
library(tidyverse)

Loading required package: DBI
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.2     ✔ dplyr   0.7.4
✔ tidyr   0.8.0     ✔ stringr 1.3.0
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
data_dir <- file.path("..", "data")
sql_dir <- file.path("..", "sql")

In [3]:
drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname = "mimic")
dbSendQuery(con, "set search_path=echo,public,mimiciii;")

<PostgreSQLResult>

In [4]:
sql <-
"
with mort_0 as (
    select echo,
        sum(case when mort_28_day = 1 then 1 else null end) as death,
        count(*) as total
    from merged_data
    where vaso = 1
    group by echo
)

, mort as (
    select echo, death, total, death / 1.0 / total as mortality
    from mort_0
)

select * from mort
"

In [5]:
mortality <- dbGetQuery(con, sql)
mortality

echo,death,total,mortality
0,368,839,0.4386174
1,400,1228,0.3257329


In [6]:
mort_fmt <- mortality %>%
mutate(echo = case_when(
    echo == 0 ~ "Non_echo",
    echo == 1 ~ "Echo"
)) %>%
select(echo, mortality) %>%
spread(echo, mortality) %>%
mutate(Method = "Raw") %>%
select(Method, Echo, Non_echo)
mort_fmt

Method,Echo,Non_echo
Raw,0.3257329,0.4386174


In [7]:
data.table::fwrite(mort_fmt, file.path(data_dir, "mort_raw_subgroup_vaso.csv"))

In [8]:
mort_ps <- data.table::fread(file.path(data_dir, "mort_ps_subgroup_vaso.csv"))

In [9]:
mort_cmb <- rbind(mort_fmt, mort_ps) %>% setNames(c("Method", "Echo", "Non-Echo"))
mort_cmb

Method,Echo,Non-Echo
Raw,0.3257329,0.4386174
PS Match,0.3325243,0.4368932


In [10]:
data.table::fwrite(mort_cmb, file.path(data_dir, "mortality_subgroup_vaso.csv"))

In [11]:
dbDisconnect(con)
dbUnloadDriver(drv)

[1] TRUE

[1] TRUE